In [1]:
import pandas as pd
import numpy as np
import random
import math
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from matplotlib import pylab as plt
from FinalID3 import *

In [2]:
targetAttribute = 'Poisonous/Edible'
targetValue = []
predictedList= []
# maxGainAttr:  spore-print-color    if  maxInfoGain > 0.144354902043, we get the data classified incorrectly
minimalGainThreshold =0.144
defaultTargetValue = 'p'
rawDataSet = 'MushroomDataSet_Before_PreProcessing.xlsx'
processedDataSet = 'MushroomDataSet_After_PreProcessing.xlsx'
columnToProcess = 'stalk-root'
res = {}
rfTargetValue = []
numberOfTrees = 10

In [3]:
datasetAfterProcessing = pd.read_excel(processedDataSet)
X = datasetAfterProcessing.drop(labels=targetAttribute, axis=1)
y = datasetAfterProcessing[targetAttribute]

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2 ,random_state = 42)



In [4]:
for i in range(numberOfTrees):
    random_state = random.randint(0,numberOfTrees/2)
    X_train_rf,X_test_rf,Y_train_rf,Y_test_rf = train_test_split(X_train,y_train,test_size = 0.9 ,random_state = random_state)
    X_train_rf[targetAttribute] = Y_train_rf
    targetValue = initializeList(X_train_rf)
    targetDict = createTree(X_train_rf)
    rfTargetValue.append(targetDict)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [ ]:

def depth(d, level=1):
    print(level)
    if not isinstance(d, dict) or not d:
        return level
    return max(depth(d[k], level + 1) for k in d)

In [ ]:
x= rfTargetValue[1] 
a = depth(x)

In [5]:
def predictTarget_noRecursion(targetDict,test):
    for k,v in targetDict.items():
        keyList = k.split('=')
        col = keyList[0].strip()
        val = keyList[1].strip()
        if(test[col].iloc[0] == val):
            if k in targetDict:
                temp = targetDict.get(k)
                if(type(temp) == dict):
                    return temp
                else:
                    return v

In [6]:
def myPredict_noRecursion(rfTargetValue, xy_train, xy_test):
    predictedList=[]
    for i in list(range(0,len(xy_train))):
        temp = []
        for tree in rfTargetValue: 
            
            predictedVal = predictTarget_noRecursion(tree,xy_train.iloc[[i]])
            while(type(predictedVal)==dict):
                predictedVal = predictTarget_noRecursion(predictedVal,xy_train.iloc[[i]])
                
            temp.append(predictedVal)
        
        tempDf= pd.DataFrame(columns= ["target"])
        tempDf["target"] = temp
        tempDf["demo"] = [1]*len(temp)
        grouped = tempDf.groupby("target")
        x= grouped.count()
        predictedVal = x['target'][0]
        print(i," : ",predictedVal)
        predictedList.append(predictedVal)
    print(len(predictedList))
    printMatrix(xy_test ,predictedList,list(targetValue))

In [7]:
myPredict_noRecursion(rfTargetValue,X_test,y_test) #FOR TESTING DATA

KeyError: 'target'

In [ ]:
def myPredict(targetDict, xy_train, xy_test):
    predictedList=[]
    print(len(predictedList))
    for i in list(range(0,len(xy_train))):
        predictedVal = predictTarget(targetDict,xy_train.iloc[[i]])
        predictedList.append(predictedVal)
    print(len(predictedList))
    printMatrix(xy_test ,predictedList,list(targetValue))

In [ ]:
def myPredict1(rfTargetValue, xy_train, xy_test):
    predictedList=[]
    for i in list(range(0,len(xy_train))):
        temp = []
        count = 1
        for tree in rfTargetValue: 
            print(count)
            predictedVal = predictTarget(tree,xy_train.iloc[[i]])
            temp.append(predictedVal)
            count = count+1
        
        ######
        #to find majority value
        tempDf= pd.DataFrame(columns= ["target"])
        tempDf["target"] = temp
        tempDf["demo"] = [1]*len(temp)
        grouped = tempDf.groupby("target")
        x= grouped.count()
        #######
        
        predictedVal = x['demo'][0] # it will have either 'p' or 'e'
        predictedList.append(predictedVal)
    print(len(predictedList))
    printMatrix(xy_test ,predictedList,list(targetValue))

In [ ]:
myPredict1(rfTargetValue,X_test,y_test) #FOR TESTING DATA

In [ ]:
myPredict1(rfTargetValue,X_train,y_train) #FOR TRAINING DATA

In [ ]:
'''
for i in list(range(0,len(X_test_rf))):
    predictedVal = predictTarget(targetDict,X_test_rf.iloc[[i]])
    predictedList.append(predictedVal)
'''

In [ ]:
#printMatrix(Y_test_rf ,predictedList,list(targetValue))

In [ ]:
#cm = confusion_matrix(y_test, pd.Series(predictedList), labels)

In [ ]:
#THIS IS JUST FOR DECISION TREE TESTING SO DON'T NEED TO INCLUDE IN RF FINAL FILE
datasetforProcessing = pd.read_excel(rawDataSet)
x = datasetforProcessing.iloc[:, 11].values
df1 = datasetforProcessing.fillna(datasetforProcessing[columnToProcess].value_counts().index[0])
writer = pd.ExcelWriter(processedDataSet, engine='xlsxwriter')
df1.to_excel(writer, sheet_name='Sheet1')
writer.save()
datasetAfterProcessing = pd.read_excel(processedDataSet)
X = datasetAfterProcessing.drop(labels=targetAttribute, axis=1)
y = datasetAfterProcessing[targetAttribute]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state= 42)
X_train[targetAttribute] = y_train
    ####################################################################################################
    ##################### Creating the distinct values for the Target Value ############################
targetValue = initializeList(X_train)
    ####################################################################################################
    ###################### Tree Builder by calculating the entropy and Information gain Recursively#####
targetDict = createTree(X_train)

In [ ]:
myPredict(targetDict,X_test,y_test) #TESTED DECISION TREE for testing data

In [ ]:
#TESTED DECISION TREE FOR training data
X_train.drop(targetAttribute,axis=1)
myPredict(targetDict,X_train,y_train)

In [15]:
df= pd.DataFrame({'name' : ['a', 'a', 'b','b','b', 'd'], 'hi' : [30,40,30,10,10,20]})

In [16]:
g = df.groupby('name')

In [17]:
x= g.count()
x

,hi
name,
a,2
b,3
d,1


In [29]:
#USUFUL TO GET MAJORITY VALUE
type(x)

max_hi = list(x['hi'])
index = max_hi.index(max(max_hi))

indexesVal = x.index.values
op =indexesVal[index]
print(op)

b


In [27]:
max_hi.index(3)

1